In [3]:
from Data.RecSys2022 import RecSys2022URMType
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from run_algorithm import test_algorithm, run_algorithm

model_class = MultiThreadSLIM_SLIMElasticNetRecommender
model_params = {'topK': 25000, 'l1_ratio': 1.0, 'alpha': 2e-4, 'workers': 6}

test_algorithm(model_class, model_params,
               urm_type=RecSys2022URMType.ONE_INTERACTED, dummies=False)

model_class = RP3betaRecommender
model_params = {'topK': 80, 'alpha': 0.719514,
                'beta': 0.229898, 'implicit': True, 'normalize_similarity': True}

test_algorithm(model_class, model_params,
               urm_type=RecSys2022URMType.ONE_INTERACTED, dummies=False)


Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria ONE_INTERACTED...
EvaluatorHoldout: Ignoring 779 ( 1.9%) Users that have less than 1 test interactions
Dataset loaded successfully with URM type: ONE_INTERACTED
Fitting model MultiThreadSLIM_SLIMElasticNetRecommender...
Fit params: {'topK': 25000, 'l1_ratio': 1.0, 'alpha': 0.0002, 'workers': 6}


  5%|▌         | 1056/19630 [00:30<03:15, 95.11it/s]

Computing evaluation...


100%|█████████▉| 19624/19630 [01:02<00:00, 313.73it/s]


EvaluatorHoldout: Processed 40850 (100.0%) in 14.79 sec. Users per second: 2762
MAP@10: 0.0311005
Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria ONE_INTERACTED...
EvaluatorHoldout: Ignoring 728 ( 1.7%) Users that have less than 1 test interactions
Dataset loaded successfully with URM type: ONE_INTERACTED
Fitting model RP3betaRecommender...
Fit params: {'topK': 80, 'alpha': 0.719514, 'beta': 0.229898, 'implicit': True, 'normalize_similarity': True}
RP3betaRecommender: Similarity column 19630 (100.0%), 4556.56 column/sec. Elapsed time 4.31 sec
Computing evaluation...
EvaluatorHoldout: Processed 40901 (100.0%) in 14.19 sec. Users per second: 2882
MAP@10: 0.0306672


In [5]:
from hybrid import Hybrid

model_class = Hybrid
model_params = {}

test_algorithm(model_class, model_params,
               urm_type=RecSys2022URMType.ONE_INTERACTED, dummies=False)

Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria ONE_INTERACTED...
EvaluatorHoldout: Ignoring 732 ( 1.8%) Users that have less than 1 test interactions
Dataset loaded successfully with URM type: ONE_INTERACTED
RP3betaRecommender: Similarity column 19630 (100.0%), 4511.16 column/sec. Elapsed time 4.35 sec




100%|█████████▉| 19624/19630 [02:13<00:00, 146.74it/s]









































































































































































































































































































































































































































Fitting model Hybrid...
Fit params: {}
Computing evaluation...


EvaluatorHoldout: Processed 40897 (100.0%) in 16.23 sec. Users per second: 2520
MAP@10: 0.0299327


In [1]:
# importing this library so the seed stays the same
import os
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample, split_train_in_two_percentage_user_wise
import Utils.not_random
from Data.RecSys2022 import RecSys2022, RecSys2022URMType
from Data_manager.split_functions.split_train_validation_random_holdout import (
    split_train_in_two_percentage_global_sample,
    split_train_in_two_percentage_user_wise)
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from hybrid import Hybrid
import tqdm

# building the dataset
# later we can call the build method to get different URM
dataset = RecSys2022()
dataset.build(type=RecSys2022URMType.ONE_INTERACTED)

urm = dataset.get_urm()
icm = dataset.get_icm()

urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])

Tensorflow is not available
Setting seed random library, os and numpy seed to 18
Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria ONE_INTERACTED...
EvaluatorHoldout: Ignoring 773 ( 1.9%) Users that have less than 1 test interactions


In [2]:
epsilons = [x/10 for x in range(0, 11)]

maps = []
results = []
highest_map = 0

hybrid_recommender = Hybrid(urm_train, icm)

with open("hybrid_results.txt", "w") as file:  
    file.write('index,epsilon,map,highest_map\n')
    file.flush()
    
    for epsilon in tqdm.tqdm(epsilons):
        hybrid_recommender.fit(norm=2, epsilon=epsilon)
        
        result_df, _ = evaluator_validation.evaluateRecommender(hybrid_recommender)

        map_value = result_df.loc[10]["MAP"]
        
        highest_map = max(highest_map, map_value)
        
        maps.append(map_value)
        results.append(result_df)
        file.write(f'{len(maps)},{epsilon},{map_value},{highest_map}\n')
        file.flush()
    
    #print('Epislon: {:.2f} - MAP: {:.8f}'.format(epsilon, map_value))

RP3betaRecommender: Similarity column 19630 (100.0%), 4607.52 column/sec. Elapsed time 4.26 sec


  0%|          | 0/11 [00:00<?, ?it/s]

EvaluatorHoldout: Processed 19000 (46.5%) in 5.14 min. Users per second: 62
EvaluatorHoldout: Processed 40856 (100.0%) in 9.97 min. Users per second: 68


  9%|▉         | 1/11 [09:58<1:39:43, 598.40s/it]

EvaluatorHoldout: Processed 22000 (53.8%) in 5.06 min. Users per second: 73
EvaluatorHoldout: Processed 40856 (100.0%) in 9.48 min. Users per second: 72


...

EvaluatorHoldout: Processed 22000 (53.8%) in 5.09 min. Users per second: 72
EvaluatorHoldout: Processed 40856 (100.0%) in 9.26 min. Users per second: 74


 64%|██████▎   | 7/11 [1:06:15<37:28, 562.23s/it]

EvaluatorHoldout: Processed 22000 (53.8%) in 5.07 min. Users per second: 72
EvaluatorHoldout: Processed 40856 (100.0%) in 9.28 min. Users per second: 73


 73%|███████▎  | 8/11 [1:15:46<28:25, 568.36s/it]


KeyboardInterrupt: 

In [4]:
hybrid_recommender = Hybrid(urm, icm)

hybrid_recommender.fit(norm=2, epsilon=0.5)

RP3betaRecommender: Similarity column 19630 (100.0%), 4023.04 column/sec. Elapsed time 4.88 sec


100%|█████████▉| 19624/19630 [01:50<00:00, 178.22it/s]


In [7]:
import pandas as pd

def create_submission(dataset: RecSys2022, model):
    user_ids = dataset.get_target_ids()
    item_original_ID_to_index = dataset.item_original_ID_to_index
    user_original_ID_to_index = dataset.user_original_ID_to_index

    predicts = []
    item_index_to_original_ID = dict(
        (v, k) for k, v in item_original_ID_to_index.items())

    for user_id in user_ids:
        masked_user_id = user_original_ID_to_index[user_id]
        recommended_items = model.recommend(masked_user_id, cutoff=10)

        unmasked_items = []
        for item_id in recommended_items:
            unmasked_items.append(item_index_to_original_ID[item_id])

        value = " ".join(map(str, unmasked_items))
        predicts.append(value)

    # dataframe with user_ids and recommended items
    return pd.DataFrame({'user_id': user_ids, 'item_list': predicts})

In [8]:
def save_submission(dataset: RecSys2022, model, filename):
    # check if model_results folder exists
    if not os.path.exists('model_results'):
        os.makedirs('model_results')
        
    submission = create_submission(dataset, model)
    submission.to_csv(os.path.join('model_results', filename), index=False)

In [9]:
save_submission(dataset, hybrid_recommender, 'hybrid_best.csv')

ValueError: Norm 2 of item weights for recommender 2 is zero. Avoiding division by zero